In [1]:
import sys
sys.path.append('/home/sarju/.local/lib/python3.12/site-packages')

In [14]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [15]:
CAFire = pd.read_csv("cleanedCAFire.csv", index_col="_id", low_memory=False)

In [16]:
CAFire = CAFire.replace(' ', np.nan)
CAFire = CAFire.replace("Unknown", np.nan)
CAFire.dropna(subset=['* Roof Construction',
       '* Eaves', '* Vent Screen', '* Exterior Siding', '* Window Pane',
       '* Deck/Porch On Grade', '* Deck/Porch Elevated',
       '* Patio Cover/Carport Attached to Structure',
       '* Fence Attached to Structure', 'Distance - Propane Tank to Structure',
       'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT'])

,OBJECTID,* Damage,* Street Number,* Street Name,"* Street Type (e.g. road, drive, lane, etc.)",* City,Zip Code,* CAL FIRE Unit,County,* Incident Name,...,* Patio Cover/Carport Attached to Structure,* Fence Attached to Structure,Distance - Propane Tank to Structure,Distance - Residence to Utility/Misc Structure &gt; 120 SQFT,Fire Name (Secondary),GLOBALID,Latitude,Longitude,x,y
_id,,,,,,,,,,,,,,,,,,,,,
91,91,Destroyed (>50%),0.0,Butte Creek,Road,Adin,NaN,LMU,Lassen,Gold,...,No Patio Cover/Carport,No Fence,0-10,<30',NaN,eb7cf382-852c-4fbf-8d89-8d1f7ddd5631,41.082369,-120.857665,-1.345381e+07,5.024499e+06
118,118,No Damage,900.0,Ash Valley,Road,Adin,NaN,LMU,Lassen,Gold,...,Combustible,No Fence,11-20,>50',NaN,d039605d-42cc-4059-a8a0-8d9d65a68b5c,41.117074,-120.750173,-1.344185e+07,5.029625e+06
144,144,Minor (10-25%),7091.0,Bluff,Street,Banning,NaN,RRU,Riverside,Apple,...,No Patio Cover/Carport,No Fence,>30,<30',NaN,b51c8af6-82a4-4d57-b69d-782d043f6a05,34.007510,-116.911264,-1.301450e+07,4.029810e+06
145,145,Major (26-50%),8001.0,Bluff,Street,Banning,NaN,RRU,Riverside,Apple,...,Combustible,No Fence,21-30,>50',NaN,ac2d80f7-3ca7-494f-9cd7-4bd13dee5a41,34.005347,-116.911414,-1.301452e+07,4.029520e+06
147,147,Major (26-50%),8201.0,Sing,Road,Banning,NaN,RRU,Riverside,Apple,...,No Patio Cover/Carport,No Fence,0-10,>50',NaN,340e8860-8f7b-4113-baef-4f1a73b3fc74,34.004802,-116.911887,-1.301457e+07,4.029447e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99782,100773,No Damage,7355.0,Balcony canyon,Road,Somis,93066.0,VNC,Ventura,Mountain,...,Combustible,Combustible,>30,>50',NaN,3d3b3ccc-b360-4cb3-8731-63c6702c4958,34.305709,-118.963356,-1.324294e+07,4.069926e+06
99947,100938,Destroyed (>50%),210.0,Avocado,Place,Camarillo,93010.0,VNC,Ventura,Mountain,...,Non Combustible,No Fence,>30,>50',NaN,1a3a6e26-135c-46f2-a1b7-96d9a122e9f8,34.234915,-119.080070,-1.325593e+07,4.060389e+06
99961,100952,Major (26-50%),150.0,Avocado,Place,Camarillo,93010.0,VNC,Ventura,Mountain,...,No Patio Cover/Carport,Combustible,>30,<30',NaN,2805e90e-88c8-4973-b0cc-325c22b64587,34.233714,-119.079874,-1.325591e+07,4.060227e+06


In [5]:
CAFire.columns

Index(['OBJECTID', '* Damage', '* Street Number', '* Street Name',
       '* Street Type (e.g. road, drive, lane, etc.)', '* City', 'Zip Code',
       '* CAL FIRE Unit', 'County', '* Incident Name',
       'Incident Number (e.g. CAAEU 123456)', 'Incident Start Date',
       'If Affected 1-9% - Where did fire start?',
       'If Affected 1-9% - What started fire?',
       'Structure Defense Actions Taken', '* Structure Type',
       'Structure Category', '# Units in Structure (if multi unit)',
       '# of Damaged Outbuildings < 120 SQFT',
       '# of Non Damaged Outbuildings < 120 SQFT', '* Roof Construction',
       '* Eaves', '* Vent Screen', '* Exterior Siding', '* Window Pane',
       '* Deck/Porch On Grade', '* Deck/Porch Elevated',
       '* Patio Cover/Carport Attached to Structure',
       '* Fence Attached to Structure', 'Distance - Propane Tank to Structure',
       'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT',
       'Fire Name (Secondary)', 'GLOBALID', 'L

In [6]:
x = pd.get_dummies(CAFire[['* Roof Construction',
       '* Eaves', '* Vent Screen', '* Exterior Siding', '* Window Pane',
       '* Deck/Porch On Grade', '* Deck/Porch Elevated',
       '* Patio Cover/Carport Attached to Structure',
       '* Fence Attached to Structure', 'Distance - Propane Tank to Structure',
       'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT']])
y = pd.get_dummies(CAFire["* Damage"])

In [7]:
# split the testing and training dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [8]:
x.columns

Index(['* Roof Construction_Asphalt', '* Roof Construction_Combustible',
       '* Roof Construction_Concrete', '* Roof Construction_Fire Resistant',
       '* Roof Construction_Metal', '* Roof Construction_No Deck/Porch',
       '* Roof Construction_Other', '* Roof Construction_Tile',
       '* Roof Construction_Wood', '* Eaves_Enclosed', '* Eaves_No Eaves',
       '* Eaves_Not Applicable', '* Eaves_Unenclosed', '* Vent Screen_21-30',
       '* Vent Screen_>30', '* Vent Screen_Deck Elevated',
       '* Vent Screen_Mesh Screen <= 1/8""',
       '* Vent Screen_Mesh Screen > 1/8""', '* Vent Screen_No Vents',
       '* Vent Screen_Screened', '* Vent Screen_Unscreened',
       '* Exterior Siding_Combustible', '* Exterior Siding_Fire Resistant',
       '* Exterior Siding_Ignition Resistant', '* Exterior Siding_Metal',
       '* Exterior Siding_Other', '* Exterior Siding_Stucco Brick Cement',
       '* Exterior Siding_Stucco/Brick/Cement', '* Exterior Siding_Vinyl',
       '* Exterior Siding

In [9]:
# Testing for variables using a LinearRegression()

rfe = RFE(LinearRegression(), n_features_to_select=1, verbose=1)
rfe.fit(x_train, y_train)
rfe_features = sorted(zip(x.columns, rfe.ranking_), key=lambda x: x[1])
for feature, rank in rfe_features:
    print(f"Feature: {feature}, Rank: {rank}")

Fitting estimator with 58 features.
Fitting estimator with 57 features.
Fitting estimator with 56 features.
Fitting estimator with 55 features.
Fitting estimator with 54 features.
Fitting estimator with 53 features.
Fitting estimator with 52 features.
Fitting estimator with 51 features.
Fitting estimator with 50 features.
Fitting estimator with 49 features.
Fitting estimator with 48 features.
Fitting estimator with 47 features.
Fitting estimator with 46 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 38 features.
Fitting estimator with 37 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 fe

In [5]:
CAFire_RandomForest = CAFire.copy()

In [17]:
x = pd.get_dummies(CAFire[['* Roof Construction',
       '* Eaves', '* Vent Screen', '* Exterior Siding', '* Window Pane',
       '* Deck/Porch On Grade', '* Deck/Porch Elevated',
       '* Patio Cover/Carport Attached to Structure',
       '* Fence Attached to Structure', 'Distance - Propane Tank to Structure',
       'Distance - Residence to Utility/Misc Structure &gt; 120 SQFT']])
y = CAFire["* Damage"]

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [24]:
# Test number of n_estimators needed for Random Forest using cross-validation
# n_estimators tested : 10 - 500
search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=0),
    param_grid = {'n_estimators': [225, 250, 275]},
    cv = 3,
    scoring="accuracy",
)
search.fit(x_train, y_train)

print("Best n_estimators:", search.best_params_)

Best n_estimators: {'n_estimators': 250}


In [21]:
# Testing for variables using Random Forest
rfe = RFE(RandomForestRegressor(n_estimators=500, random_state=0), n_features_to_select=1)
rfe.fit(x_train, y_train)
rfe_features = sorted(zip(x.columns, rfe.ranking_), key=lambda x: x[1])
for feature, rank in rfe_features:
    print(f"Feature: {feature}, Rank: {rank}")

Feature: * Exterior Siding_Combustible, Rank: 1
Feature: * Exterior Siding_Ignition Resistant, Rank: 2
Feature: * Roof Construction_Fire Resistant, Rank: 3
Feature: * Eaves_Unenclosed, Rank: 4
Feature: * Eaves_Enclosed, Rank: 5
Feature: * Eaves_No Eaves, Rank: 6
Feature: * Fence Attached to Structure_No Fence, Rank: 7
Feature: * Exterior Siding_Fire Resistant, Rank: 8
Feature: * Window Pane_Multi Pane, Rank: 9
Feature: * Vent Screen_No Vents, Rank: 10
Feature: * Roof Construction_Metal, Rank: 11
Feature: * Exterior Siding_Wood, Rank: 12
Feature: * Patio Cover/Carport Attached to Structure_No Patio Cover/Carport, Rank: 13
Feature: * Deck/Porch Elevated_Wood, Rank: 14
Feature: * Deck/Porch Elevated_No Deck/Porch, Rank: 15
Feature: * Deck/Porch On Grade_Wood, Rank: 16
Feature: * Patio Cover/Carport Attached to Structure_Combustible, Rank: 17
Feature: * Deck/Porch On Grade_No Deck/Porch, Rank: 18
Feature: * Roof Construction_Asphalt, Rank: 19
Feature: * Exterior Siding_Stucco Brick Cement,

In [26]:
# Test the RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=250, random_state=0)
rf_model.fit(x_train, y_train)
rf_predictor = rf_model.predict(x_test)
accuracy = accuracy_score(y_test, rf_predictor)

print(f"Accuracy: {accuracy}")

Accuracy: 0.8387708270976754
